In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python '/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/EDA.py'

In [ ]:
!pip install karateclub
!pip install node2vec
!pip install dgl

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numba
from numba import jit,njit, cuda
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp
import networkx as nx
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score, f1_score, recall_score,precision_score, accuracy_score,average_precision_score,precision_recall_curve,auc
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from karateclub import DeepWalk
from node2vec import Node2Vec

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [ ]:
'''
here, the hyG creation starts
1. at frist we read the text file "new_graph_data_1.txt". This text file is created by running ESPF algorithm
2. Then we create a dictionary 'DICT' that has 'drug' as key and drug's 'substructures' as values 
'''
formattedNodes = []
smileFile = open("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/new_graph_data_645_ESPF_5.txt", 'r')
names = []

for line in smileFile:
    line = line.strip()
    words = line.split(" ")
    name = words[0]
    del words[0]
    t=tuple(words)
    names.append(name)
    formattedNodes.append(t)
      
# Make the dictionary of the Drug IDs: Smiles tokens    
DICT = dict()  
for i in range(0, len(names)):
    DICT[names[i]] = formattedNodes[i]  
LEN=len(DICT)      

In [ ]:
'''
converting to sparse matrix
'''
df=len(DICT)
from scipy.sparse import coo_matrix
nl=coo_matrix((df, df))
nl.setdiag(1)
#nl.toarray()
values = nl.data
indices = np.vstack((nl.row, nl.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = nl.shape
nl=torch.sparse_coo_tensor(i, v, torch.Size(shape))

In [ ]:
'''
Then we are using the DICT to create a hyG
'''
drug_list = [] # list of drug - translates drug name into index
chemicalsub_list = [] # list of chemicalsub - translates chemicalsub number into index
drug_chemicalsub = {} # dict of int:list - drug idx i wrote chemicalsubs with idx [j, k, l, .. ]
#chemicalsub_drug = {} # dict of int:list - chemicalsub idx i were written by drugs with idx [j, k, l, ..]
numba.njit(target="cuda")
def function(DICT):
  for drug in DICT.keys() :
      chemicalsubs = DICT[drug]
      if drug not in drug_list :
          drug_list.append(drug)
      idx = drug_list.index(drug)
      if idx not in drug_chemicalsub :
          drug_chemicalsub[idx]=[]   
      
      translated_p = []
    
      for chemicalsub in chemicalsubs :
          if chemicalsub not in chemicalsub_list :
              chemicalsub_list.append(chemicalsub)
          p_idx = chemicalsub_list.index(chemicalsub)
          translated_p.append(p_idx) # translate chemicalsub number into index
    
      drug_chemicalsub[idx]=translated_p
  return chemicalsub_list,drug_list,drug_chemicalsub    
function1=jit(parallel=True) (function)
chemicalsub_list,drug_list,drug_chemicalsub = function1(DICT)

'''
chemicalsub : corresponds to hypernode
citing : corresponds to hyperedge(drug)
'''

chemicalsub_citing = [] # list of [chemicalsub drug]
n_chemicalsub = len(chemicalsub_list)
n_hedge = len(drug_list)

for drug in drug_chemicalsub.keys():
    chemicalsubs = drug_chemicalsub[drug]
    #if len(chemicalsubs) == 1 :
        #continue # remove one-cited chemicalsubs
    for chemicalsub in chemicalsubs : 
        chemicalsub_citing.append([chemicalsub, drug])
       
chemicalsub_drug = torch.LongTensor(chemicalsub_citing)
data_dict = {
        ('node', 'in', 'edge'): (chemicalsub_drug[:,0], chemicalsub_drug[:,1]),        
        ('edge', 'con', 'node'): (chemicalsub_drug[:,1], chemicalsub_drug[:,0])
    }

'''
finally passing the data_dict to construct hyG
'''
#hyG = dgl.heterograph(data_dict)


lst=[]
for i in chemicalsub_citing:
  lst.append(i[0])
s=set(lst)
s=len(s)  
num_nodes_dict = {'edge': LEN,'node':s}
hyG = dgl.heterograph(data_dict,num_nodes_dict=num_nodes_dict)
rows=n_chemicalsub
columns=n_hedge
'''
reading the feature (one hot coding) for drugs (edges)
'''

#drug_X=np.loadtxt("/content/drive/MyDrive/My Drive/Colab Notebooks/DNA-Protein/DNA_features_one_hot_coding.txt", dtype=int) #if you want to use 167 bits representation of each drug as feature read: features_bit.txt    
drug_X=nl

v_feat=coo_matrix((rows, 128))
v_feat.setdiag(1)
#nl.toarray()
values = v_feat.data
indices = np.vstack((v_feat.row, v_feat.col))
i = torch.LongTensor(indices)
v = torch.FloatTensor(values)
shape = v_feat.shape
v_feat=torch.sparse_coo_tensor(i, v, torch.Size(shape))

hyG.ndata['h'] = {'edge' : drug_X.type('torch.FloatTensor'), 'node' : v_feat.type('torch.FloatTensor')}
e_feat = drug_X.type('torch.FloatTensor')
v_feat=v_feat.type('torch.FloatTensor')

hyG

Graph(num_nodes={'edge': 645, 'node': 555},
      num_edges={('edge', 'con', 'node'): 7547, ('node', 'in', 'edge'): 7547},
      metagraph=[('edge', 'node', 'con'), ('node', 'edge', 'in')])

In [ ]:
'''
Don't run second time
'''
f = open( '/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/rows_drug_645_kmer_9.txt', 'w' )
f.write( 'num of rows = ' + str(rows) + '\n')
f.close()

torch.save(chemicalsub_drug, '/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/hyG_drug_645_kmer_9.pt')

In [ ]:
chemicalsub_drug = torch.load('/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/hyG_drug_645_kmer_9.pt')
data_dict = {
        ('node', 'in', 'edge'): (chemicalsub_drug[:,0], chemicalsub_drug[:,1]),        
        ('edge', 'con', 'node'): (chemicalsub_drug[:,1], chemicalsub_drug[:,0])
    }
'''
finally passing the data_dict to construct hyG
'''
hyG = dgl.heterograph(data_dict)
n_chemicalsub=14002
rows=n_chemicalsub
n_hedge=LEN
columns=n_hedge
'''
reading the feature (one hot coding) for drugs (edges)
'''
#drug_X=np.loadtxt("/content/drive/MyDrive/My Drive/Colab Notebooks/DNA-Protein/DNA_features_one_hot_coding.txt", dtype=int) #if you want to use 167 bits representation of each drug as feature read: features_bit.txt    
drug_X=nl
hyG.ndata['h'] = {'edge' : torch.tensor(drug_X).type('torch.FloatTensor'), 'node' : torch.ones(rows, 128)}
e_feat = torch.tensor(drug_X).type('torch.FloatTensor')
v_feat = torch.ones(rows, 128)

In [ ]:
hyG

Graph(num_nodes={'edge': 645, 'node': 555},
      num_edges={('edge', 'con', 'node'): 7547, ('node', 'in', 'edge'): 7547},
      metagraph=[('edge', 'node', 'con'), ('node', 'edge', 'in')])

In [ ]:
"""
*** for TOWSIDES (645) ddi data *******
The main regular graph creation (mat) from drugbank data, that will be used for train-test purpose.
1. first we are creating a 2D matrix 'mat'
2. writing the edge information in a text file
3. reading the saved text file and extracting the src and dst node from that for each edge.
These src, dst info will be used in DGL to create a dgl type graph
"""

#1:
df=pd.read_csv("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/TDC_TWOSIDES_useable.csv",header=None) 
cols=['Drug1_ID','Drug1','Drug2_ID','Drug2']
df1=pd.read_csv("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/TDC_TWOSIDES.csv",usecols=cols) 
df1 = df1.drop_duplicates(subset=cols)
df1.reset_index(drop=True, inplace=True)

D={}
for i in range (len (df)):
    D[df[0][i]]=i

drug1_lst=df1['Drug1_ID'].tolist()
drug2_lst=df1['Drug2_ID'].tolist()
LEN1=len(df1['Drug1_ID'])
mat = np.zeros((LEN, LEN))
numba.njit(target="cuda")
def func(D,mat):
  for i in range (LEN1):
          for drug, id in D.items():
              if drug==drug1_lst[i]:
                  x=id 
              elif drug==drug2_lst[i]:
                  y=id 
              elif x!=-1 and y!=-1:
                break      
          print(x,y)
          mat[x,y]=mat[y,x]=1
          x=y=-1
  return mat                     
function1=jit(parallel=True) (func)
mat = function1(D,mat)

#np.count_nonzero(mat == 1)    
'''
#2:
with open("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/edge_list_regular_graph_1706.txt", "w") as file_prime:
  for i in range (len(mat)):
    for j in range (len(mat)):
      if mat[i,j] == 1.0:
        stra=str(i)
        strb=str(j)
        strc=stra+'\t'+strb+'\n'
        file_prime.write(strc)


#3:
src=[]
dst=[]

with open("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/edge_list_regular_graph_1706.txt") as fp:
        for i,line in enumerate (fp):
            info = line.strip().split()
            src.append(info[0])
            dst.append(info[1])

src=np.asarray(src, dtype=np.int64) #to use in the data prerpocessing
dst=np.asarray(dst, dtype=np.int64) #to use in the data prerpocessing
'''

In [ ]:
len(df1)

63473

In [ ]:
#3:
src=[]
dst=[]

with open("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/edge_list_regular_graph_645.txt") as fp:
        for i,line in enumerate (fp):
            info = line.strip().split()
            src.append(info[0])
            dst.append(info[1])

src=np.asarray(src, dtype=np.int64) #to use in the data prerpocessing
dst=np.asarray(dst, dtype=np.int64) #to use in the data prerpocessing

In [ ]:
np.count_nonzero(mat == 1) 

126946

In [ ]:
"""
*** for old 824 ddi data *******
The main regular graph creation (mat) from drugbank data, that will be used for train-test purpose.
1. first we are creating a 2D matrix 'mat'
2. writing the edge information in a text file
3. reading the saved text file and extracting the src and dst node from that for each edge.
These src, dst info will be used in DGL to create a dgl type graph
"""

#1:
df=pd.read_csv("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/drug_and_smiles_824.csv") 
mat = np.zeros((824, 824))
df1=pd.read_csv("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/drugID_to_Integer_Mapping_824.txt", header=None, sep=' ')
D={}
for i in range (len (df1)):
    D[df1[1][i]]=df1[0][i]
for i in range (len(df['Interacting Drugs'])):
    SP=df['Interacting Drugs'][i]
    SP=SP.split('; ')
    SP1=list(SP)
    for j in SP1:
        for drug, id in D.items():
            if j==drug:
                mat[id,i]=mat[i,id]=1

#np.count_nonzero(mat == 1)    

#2:
with open("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/edge_list_regular_graph_824.txt", "w") as file_prime:
  for i in range (len(mat)):
    for j in range (len(mat)):
      if mat[i,j] == 1.0:
        stra=str(i)
        strb=str(j)
        strc=stra+'\t'+strb+'\n'
        file_prime.write(strc)


#3:
src=[]
dst=[]

with open("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/edge_list_regular_graph_824.txt") as fp:
        for i,line in enumerate (fp):
            info = line.strip().split()
            src.append(info[0])
            dst.append(info[1])

src=np.asarray(src, dtype=np.int64) #to use in the data prerpocessing
dst=np.asarray(dst, dtype=np.int64) #to use in the data prerpocessing


In [ ]:
"""
here, we are using src, dst info to create a DGL style graph. this graph (g) will be used for training and testing purpose.

Source: DGL
"""
from dgl.data import DGLDataset
import os
import torch.nn as nn
import torch.nn.functional as F

class KarateClubDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='karate_club')
    def process(self):
        edges_src = torch.from_numpy(src)
        edges_dst = torch.from_numpy(dst)
        self.graph = dgl.graph((edges_src, edges_dst), num_nodes=LEN)
    def __getitem__(self, i):
        return self.graph
    def __len__(self):
        return 1

dataset = KarateClubDataset()
g = dataset[0]
print(g)

Graph(num_nodes=645, num_edges=126946,
      ndata_schemes={}
      edata_schemes={})


In [ ]:
########################################## for new drugs #########################################
np.random.seed(42)
u, v = g.edges()
nids = np.arange(g.number_of_nodes())
#nids = np.random.permutation(nids)

val_size=2000

test_size = int(len(nids) * 0.05)
train_size = g.number_of_nodes() - test_size

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)

test_pos_eids = [i for i in eids if u[i]<=test_size or v[i]<=test_size]
test_pos_u, test_pos_v = u[test_pos_eids[:]], v[test_pos_eids[:]]

train_pos_eids = [i for i in eids if u[i]>test_size and v[i]>test_size]
train_pos_u, train_pos_v = u[train_pos_eids[:]], v[train_pos_eids[:]]
val_pos_u,val_pos_v=train_pos_u[:val_size], train_pos_v[:val_size]
train_pos_u, train_pos_v = train_pos_u[val_size:], train_pos_v[val_size:]


adj = sp.coo_matrix((np.ones(len(u)), (u.cpu().numpy(), v.cpu().numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)
neg_eids = np.random.choice(len(neg_u), g.number_of_edges())

test_neg_eids = [i for i in neg_eids if neg_u[i]<=test_size or neg_v[i]<=test_size]
test_neg_u, test_neg_v = neg_u[test_neg_eids[:]], neg_v[test_neg_eids[:]]
'''
if (len(test_neg_u)) != len(test_pos_u):
  if len(test_pos_u)> len(test_neg_u):
    a=len(test_pos_u)
    b=len(test_pos_u)-len(test_neg_u)
    test_pos_u=test_pos_u[:a-b]
    test_pos_v=test_pos_v[:a-b]
    
  if len(test_pos_u)< len(test_neg_u):
    a=len(test_neg_u)
    b=len(test_neg_u)-len(test_pos_u)
    test_neg_u=test_neg_u[:a-b]
    test_neg_v=test_neg_v[:a-b]  
'''

train_neg_eids = [i for i in neg_eids if neg_u[i]>test_size and neg_v[i]>test_size]
train_neg_u, train_neg_v = neg_u[train_neg_eids[:]], neg_v[train_neg_eids[:]]
val_neg_u,val_neg_v=train_neg_u[:val_size], train_neg_v[:val_size]
train_neg_u, train_neg_v = train_neg_u[val_size:], train_neg_v[val_size:]
'''
if (len(train_neg_u)) != len(train_pos_u):
  if len(train_pos_u)> len(train_neg_u):
    a=len(train_pos_u)
    b=len(train_pos_u)-len(train_neg_u)
    train_pos_u=train_pos_u[:a-b]
    train_pos_v=train_pos_v[:a-b]
    
  if len(train_pos_u)< len(train_neg_u):
    a=len(train_neg_u)
    b=len(train_neg_u)-len(train_pos_u)
    train_neg_u=train_neg_u[:a-b]
    train_neg_v=train_neg_v[:a-b]  
'''

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())# this is just the edge_list
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

print("Total number of edges: ",g.number_of_edges())
print("num of train POSITIVE edges: ",train_pos_g.number_of_edges(),", num of test POSITIVE edges: ",test_pos_g.number_of_edges(),"num of val POSITIVE edges: ",val_pos_g.number_of_edges())
print("num of train NEGATIVE edges: ",train_neg_g.number_of_edges(),", num of test NEGATIVE edges: ",test_neg_g.number_of_edges(),"num of val NEGATIVE edges: ",val_neg_g.number_of_edges())


Total number of edges:  126946
num of train POSITIVE edges:  107588 , num of test POSITIVE edges:  17358 num of val POSITIVE edges:  2000
num of train NEGATIVE edges:  114308 , num of test NEGATIVE edges:  10638 num of val NEGATIVE edges:  2000


In [ ]:
# Split edge set for training and testing
u, v = g.edges()
np.random.seed(42)
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.8)
val_size = int(len(eids) * 0.1)
train_size=len(eids)-(test_size+val_size)

#train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
val_pos_u, val_pos_v = u[eids[test_size:test_size+val_size]], v[eids[test_size:test_size+val_size]]
train_pos_u, train_pos_v = u[eids[test_size+val_size:]], v[eids[test_size+val_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges(),replace=False)
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
val_neg_u, val_neg_v = neg_u[neg_eids[test_size:test_size+val_size]], neg_v[neg_eids[test_size:test_size+val_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size+val_size:]], neg_v[neg_eids[test_size+val_size:]]

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())# this is just the edge_list
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

print("Total number of edges: ",g.number_of_edges())
print("num of train POSITIVE edges: ",train_pos_g.number_of_edges(),", num of test POSITIVE edges: ",test_pos_g.number_of_edges(),"num of val POSITIVE edges: ",val_pos_g.number_of_edges())
print("num of train NEGATIVE edges: ",train_neg_g.number_of_edges(),", num of test NEGATIVE edges: ",test_neg_g.number_of_edges(),"num of val NEGATIVE edges: ",val_neg_g.number_of_edges())

Total number of edges:  382804
num of train POSITIVE edges:  38281 , num of test POSITIVE edges:  306243 num of val POSITIVE edges:  38280
num of train NEGATIVE edges:  38281 , num of test NEGATIVE edges:  306243 num of val NEGATIVE edges:  38280


In [ ]:
'''
*************another approach
Train, test (both positive and negative) graphs creation for training and testing purpose from the dgl graph (g)
Source: DGL
'''
np.random.seed(42)
# Split edge set for training and testing
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
eids=eids.tolist()
test_size = int(len(eids) * 0.1)
val_size = int(len(eids) * 0.1)
train_size=len(eids)-(test_size+val_size)
test_size1=int(test_size/2)


T1=[]
T2=[]
numba.njit(target="cuda")
def func2(T1):
  for i in eids:
    m, n = int(u[i]), int(v[i])
    c=(m,n)
    T1.append(c)
  return T1  
function1=jit(parallel=True) (func2)
T1 = function1(T1)


numba.njit(target="cuda")
def func3(T1,T2):
  for i in T1:
    m=i[0]
    n=i[1]
    c=(m,n)
    d=(n,m)
    if c not in T2 and d not in T2:
      T2.append(c)
    if len(T2)>=test_size1:
      break  
  return T2
function1=jit(parallel=True) (func3)
T2 = function1(T1,T2)


test_eids=[]
numba.njit(target="cuda")
def func4(T2,test_eids,g):
  for i in T2:
    m= i[0]
    n=i[1]
    c=g.edge_id(m,n)
    d=g.edge_id(n,m)
    test_eids.append(c)
    test_eids.append(d)
  return test_eids  
function1=jit(parallel=True) (func4)
test_eids = function1(T2,test_eids,g)

val_eids=[]
numba.njit(target="cuda")
def func5(eids,test_eids,val_eids):
  i=0
  for j in eids:
    a=int(eids[i])
    if a not in test_eids and len(val_eids)<val_size:
      val_eids.append(a)  
    i=i+1
  return val_eids    
function1=jit(parallel=True) (func5)
val_eids = function1(eids,test_eids,val_eids)     

train_eids=[]
numba.njit(target="cuda")
def func6(eids,test_eids,val_eids,train_eids):
  i=0
  for j in eids:
    a=int(eids[i])
    if a not in test_eids and a not in val_eids and len(train_eids)<train_size:
      train_eids.append(a)  
    i=i+1
  return train_eids    
function1=jit(parallel=True) (func6)
train_eids = function1(eids,test_eids,val_eids,train_eids)     

# main one. splitting into train and test set.
test_pos_u, test_pos_v = u[test_eids], v[test_eids]
train_pos_u, train_pos_v = u[train_eids], v[train_eids]
val_pos_u, val_pos_v = u[val_eids], v[val_eids]


# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges(),replace=False)
g_=nx.from_numpy_matrix(adj_neg)
g_=dgl.from_networkx(g_)

T1=[]
T2=[]
numba.njit(target="cuda")
def func2(T1):
  for i in neg_eids:
    m, n = int(neg_u[i]), int(neg_v[i])
    c=(m,n)
    T1.append(c)
  return T1  
function1=jit(parallel=True) (func2)
T1 = function1(T1)


numba.njit(target="cuda")
def func3(T1,T2):
  for i in T1:
    m=i[0]
    n=i[1]
    c=(m,n)
    d=(n,m)
    if c not in T2 and d not in T2:
      T2.append(c)
    if len(T2)>=test_size1:
      break  
  return T2
function1=jit(parallel=True) (func3)
T2 = function1(T1,T2)


test_neg_eids=[]
numba.njit(target="cuda")
def func4(T2,test_eids,g):
  for i in T2:
    m= i[0]
    n=i[1]
    c=g.edge_id(m,n)
    d=g.edge_id(n,m)
    test_neg_eids.append(c)
    test_neg_eids.append(d)
  return test_neg_eids  
function1=jit(parallel=True) (func4)
test_neg_eids = function1(T2,test_neg_eids,g_)

val_neg_eids=[]
numba.njit(target="cuda")
def func5(neg_eids,test_neg_eids,val_neg_eids):
  i=0
  for j in neg_eids:
    a=int(neg_eids[i])
    if a not in test_neg_eids and len(val_neg_eids)<(val_size):
      val_neg_eids.append(a)  
    i=i+1
  return val_neg_eids    
function1=jit(parallel=True) (func5)
val_neg_eids = function1(neg_eids,test_neg_eids,val_neg_eids)  

train_neg_eids=[]
numba.njit(target="cuda")
def func6(neg_eids,test_neg_eids,val_neg_eids,train_neg_eids):
  i=0
  for j in neg_eids:
    a=int(neg_eids[i])
    if a not in test_neg_eids and a not in val_neg_eids and len(train_neg_eids)<(train_size):
      train_neg_eids.append(a)  
    i=i+1
  return train_neg_eids    
function1=jit(parallel=True) (func6)
train_neg_eids = function1(neg_eids,test_neg_eids,val_neg_eids,train_neg_eids)     

test_neg_u, test_neg_v = neg_u[test_neg_eids], neg_v[test_neg_eids]
train_neg_u, train_neg_v = neg_u[train_neg_eids], neg_v[train_neg_eids]
val_neg_u, val_neg_v = neg_u[val_neg_eids], neg_v[val_neg_eids]

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())# this is just the edge_list
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

print("Total number of edges: ",g_.number_of_edges())
print("num of train POSITIVE edges: ",train_pos_g.number_of_edges(),", num of test POSITIVE edges: ",test_pos_g.number_of_edges(),"num of val POSITIVE edges: ",val_pos_g.number_of_edges())
print("num of train NEGATIVE edges: ",train_neg_g.number_of_edges(),", num of test NEGATIVE edges: ",test_neg_g.number_of_edges(),"num of val NEGATIVE edges: ",val_neg_g.number_of_edges())

Total number of edges:  288434
num of train POSITIVE edges:  101558 , num of test POSITIVE edges:  12694 num of val POSITIVE edges:  12694
num of train NEGATIVE edges:  101558 , num of test NEGATIVE edges:  12694 num of val NEGATIVE edges:  12694


In [ ]:
'''
Data prep for CASTER. This part returns negative edges
'''

neg_u, neg_v
neg_u_s=[]
neg_v_s=[]

numba.njit(target="cuda")
def func(neg_u):
  for i in neg_u:
    for j,k in D.items():
      if i==k:
        neg_u_s.append(j)
        print('o')
        break
  return neg_u_s      
function1=jit(parallel=True) (func)
neg_u_s = function1(neg_u) 

numba.njit(target="cuda")
def func(neg_v):
  for i in neg_v:
    for j,k in D.items():
      if i==k:
        neg_v_s.append(j)
        print('o')
        break
  return neg_v_s      
function1=jit(parallel=True) (func)
neg_v_s = function1(neg_v)   

D_1={}
for i in range (len(df[1])):
    D_1[df[0][i]]=df[1][i]

neg_u_s,neg_v_s
neg_u_s1=[]
neg_v_s1=[]

numba.njit(target="cuda")
def func(neg_u_s):
  for i in neg_u_s:
    for j,k in D_1.items():
      if i==j:
        neg_u_s1.append(k)
        print('o')
        break
  return neg_u_s1     
function1=jit(parallel=True) (func)
neg_u_s1 = function1(neg_u_s) 

numba.njit(target="cuda")
def func(neg_v_s):
  for i in neg_v_s:
    for j,k in D_1.items():
      if i==j:
        neg_v_s1.append(k)
        print('o')
        break
  return neg_v_s1      
function1=jit(parallel=True) (func)
neg_v_s1 = function1(neg_v_s)  

LEN=len(u)
a_list_neg=[0]*LEN
len(a_list_neg)

df_re_reg = pd.DataFrame(list(zip(neg_u_s, neg_u_s1,neg_v_s,neg_v_s1,a_list_neg)),columns =['Drug1_ID', 'Drug1_SMILES','Drug2_ID','Drug2_SMILES','label'])
df_re_reg

In [ ]:
'''
Data prep for CASTER. This part returns positive edges
'''
u,v
u_s=[]
v_s=[]

numba.njit(target="cuda")
def func(u):
  for i in u:
    for j,k in D.items():
      if i==k:
        u_s.append(j)
        print('o')
        break
  return u_s      
function1=jit(parallel=True) (func)
u_s = function1(u) 

numba.njit(target="cuda")
def func(v):
  for i in v:
    for j,k in D.items():
      if i==k:
        v_s.append(j)
        print('o')
        break
  return v_s      
function1=jit(parallel=True) (func)
v_s = function1(v)   

D_1={}
for i in range (len(df[1])):
    D_1[df[0][i]]=df[1][i]

u_s,v_s
u_s1=[]
v_s1=[]

numba.njit(target="cuda")
def func(u_s):
  for i in u_s:
    for j,k in D_1.items():
      if i==j:
        u_s1.append(k)
        print('o')
        break
  return u_s1     
function1=jit(parallel=True) (func)
u_s1 = function1(u_s) 

numba.njit(target="cuda")
def func(v_s):
  for i in v_s:
    for j,k in D_1.items():
      if i==j:
        v_s1.append(k)
        print('o')
        break
  return v_s1      
function1=jit(parallel=True) (func)
v_s1 = function1(v_s)  

LEN=len(u)
a_list_pos=[1]*LEN
len(a_list_pos)

In [ ]:
df_re = pd.DataFrame(list(zip(u_s, u_s1,v_s,v_s1,a_list_pos)),columns =['Drug1_ID', 'Drug1_SMILES','Drug2_ID','Drug2_SMILES','label'])
print(len(df_re_reg))
df_re = df_re.append(df_re_reg, ignore_index=True)
df_re = df_re.sample(frac=1).reset_index(drop=True)
df_re.to_csv('/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/TDC_DrugBank_FOR_CASTER.csv',index=None)

382804


In [ ]:
'''
when u want to exclude some nodes' edge (DDI) from trainset. You can do this in one of three ways:1. using a for loop 2. using a list of those node id 
3. or directly putting those drugs' edges in 'testset (pos/neg), that are excluded form trainset 
'''
'''
test_pos_u, test_pos_v = u[test_eids], v[test_eids]
train_pos_u, train_pos_v = u[train_eids], v[train_eids]
val_pos_u, val_pos_v = u[val_eids], v[val_eids]
'''
print("Before")
print("train pos edges: ",len(train_pos_u),"test pos edges: ",len(test_pos_u),"val pos edges: ",len(val_pos_u))

import random
random.seed(42)
lst=[]
#1:
for i in range(0,644):
    lst.append(i)
lst=random.sample(lst,129)
#2:
#back=[10,501,75,224,541,365,126,696,775,615,656,622,533,339,376,690,151,360,9,387,92,412,788,770] #some nodes that u don't want to keep in trainset (pos/neg)
#for i in back:
#  lst.append(i)
train_pos_u1=[]
train_pos_v1=[]
test_pos_u1=[]
test_pos_v1=[]

for i in range(len(train_pos_u)):
  if train_pos_u[i] not in lst and train_pos_v[i] not in lst:
    train_pos_u1.append(train_pos_u[i])
    train_pos_v1.append(train_pos_v[i])
  if train_pos_u[i] in lst and train_pos_v[i] in lst:  
    test_pos_u1.append(train_pos_u[i])
    test_pos_v1.append(train_pos_v[i])


val_pos_u1=[]
val_pos_v1=[]
for i in range(len(val_pos_u)):
  if val_pos_u[i] not in lst and val_pos_v[i] not in lst:
    val_pos_u1.append(val_pos_u[i])
    val_pos_v1.append(val_pos_v[i])
  if val_pos_u[i] in lst and val_pos_v[i] in lst:
    test_pos_u1.append(val_pos_u[i])
    test_pos_v1.append(val_pos_v[i])
        
val_pos_u=torch.tensor(val_pos_u1)
val_pos_v=torch.tensor(val_pos_v1)


for i in range(len(test_pos_u)):
  if test_pos_u[i]  in lst and test_pos_v[i]  in lst:
    test_pos_u1.append(test_pos_u[i])
    test_pos_v1.append(test_pos_v[i]) 
  if test_pos_u[i] not in lst and test_pos_v[i] not in lst:
    train_pos_u1.append(test_pos_u[i])
    train_pos_v1.append(test_pos_v[i])   
test_pos_u=torch.tensor(test_pos_u1)
test_pos_v=torch.tensor(test_pos_v1)

train_pos_u=torch.tensor(train_pos_u1)
train_pos_v=torch.tensor(train_pos_v1)

print("After")
print("train pos edges: ",len(train_pos_u),"test pos edges: ",len(test_pos_u),"val pos edges: ",len(val_pos_u))

'''
test_neg_u, test_neg_v = neg_u[test_neg_eids], neg_v[test_neg_eids]
train_neg_u, train_neg_v = neg_u[train_neg_eids], neg_v[train_neg_eids]
val_neg_u, val_neg_v = neg_u[val_neg_eids], neg_v[val_neg_eids]
'''
print("Before")
print("train neg edges: ",len(train_neg_u),"test neg edges: ",len(test_neg_u),"val neg edges: ",len(val_neg_u))

################

train_neg_u1=[]
train_neg_v1=[]
test_neg_u1=[]
test_neg_v1=[]
for i in range(len(train_neg_u)):
  if train_neg_u[i] not in lst and train_neg_v[i] not in lst:
    if len(train_neg_u1)<= len(train_pos_u):
      train_neg_u1.append(train_neg_u[i])
      train_neg_v1.append(train_neg_v[i])
  if train_neg_u[i]  in lst and train_neg_v[i]  in lst:
    if len(test_neg_u1)<= len(test_pos_u):
      test_neg_u1.append(train_neg_u[i])
      test_neg_v1.append(train_neg_v[i])  


val_neg_u1=[]
val_neg_v1=[]
for i in range(len(val_pos_u)):
  if val_neg_u[i] not in lst and val_neg_v[i] not in lst:
    if len(val_neg_u1)<= len(val_pos_u):
      val_neg_u1.append(val_neg_u[i])
      val_neg_v1.append(val_neg_v[i])
  if val_neg_u[i]  in lst and val_neg_v[i]  in lst:
    if len(test_neg_u1)<= len(test_pos_u):
      test_neg_u1.append(val_neg_u[i])
      test_neg_v1.append(val_neg_v[i])  
val_neg_u=torch.tensor(val_neg_u1)
val_neg_v=torch.tensor(val_neg_v1)


for i in range(len(test_neg_u)):
  if test_neg_u[i]  in lst and test_neg_v[i]  in lst:
    if len(test_neg_u1)<=len(test_pos_u):
      test_neg_u1.append(int(test_neg_u[i]))
      test_neg_v1.append(int(test_neg_v[i]))
  if test_neg_u[i]  not in lst and test_neg_v[i]  not in lst:
    if len(train_neg_u1)<= len(train_pos_u):
      train_neg_u1.append(int(test_neg_u[i]))
      train_neg_v1.append(int(test_neg_v[i]))  
test_neg_u=torch.tensor(test_neg_u1)
test_neg_v=torch.tensor(test_neg_v1)
train_neg_u=torch.tensor(train_neg_u1)
train_neg_v=torch.tensor(train_neg_v1)
##################
print("After")
print("train neg edges: ",len(train_neg_u),"test neg edges: ",len(test_neg_u),"val neg edges: ",len(val_neg_u))

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())# this is just the edge_list
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())
val_pos_g = dgl.graph((val_pos_u, val_pos_v), num_nodes=g.number_of_nodes())
val_neg_g = dgl.graph((val_neg_u, val_neg_v), num_nodes=g.number_of_nodes())

In [ ]:
p_num_edges=val_neg_g.num_edges()
u,v=val_pos_g.edges()
remain=len(u)-p_num_edges

lst=range(0,remain)
val_pos_g.remove_edges(lst)


p_num_edges=train_neg_g.num_edges()
u,v=train_pos_g.edges()
remain=len(u)-p_num_edges

lst=range(0,remain)
train_pos_g.remove_edges(lst)

In [ ]:
#3: 
l_u=[10,75,541,126,775,656]
l_v=[501,224,365,696,615,622]
test_pos_u1=[]
test_pos_v1=[]

for i in range (len(l_u)):
  test_pos_u1.append(int(l_u[i]))
  test_pos_v1.append(int(l_v[i]))
test_pos_u=torch.tensor(test_pos_u1)
test_pos_v=torch.tensor(test_pos_v1)
  
l_u=[376,151,9,92.788]
l_v=[690,360,387,412,770]
test_neg_u1=[]
test_neg_v1=[]

for i in range (len(l_u)):
  test_neg_u1.append(int(l_u[i]))
  test_neg_v1.append(int(l_v[i]))
test_neg_u=torch.tensor(test_neg_u1)
test_neg_v=torch.tensor(test_neg_v1)

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [ ]:
print("Total number of edges: ",g.number_of_edges())
print("num of train POSITIVE edges: ",train_pos_g.number_of_edges(),", num of test POSITIVE edges: ",test_pos_g.number_of_edges(),"num of val POSITIVE edges: ",val_pos_g.number_of_edges())
print("num of train NEGATIVE edges: ",train_neg_g.number_of_edges(),", num of test NEGATIVE edges: ",test_neg_g.number_of_edges(),"num of val NEGATIVE edges: ",val_neg_g.number_of_edges())

g_for_baseline = dgl.remove_edges(g, test_eids)
#dgl.save_graphs("drive/My Drive/Colab Notebooks/Hypergraph_my/a.bin",g_for_baseline)
g_for_baseline = dgl.add_self_loop(g_for_baseline)

Total number of edges:  126946
num of train POSITIVE edges:  101558 , num of test POSITIVE edges:  12694 num of val POSITIVE edges:  12694
num of train NEGATIVE edges:  101558 , num of test NEGATIVE edges:  12694 num of val NEGATIVE edges:  12694


In [ ]:
'''
compute_loss() and compute_auc() define
'''
def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy() 
    precision, recall, thresholds = precision_recall_curve(labels, scores)
    auc_precision_recall = auc(recall, precision)
    return roc_auc_score(labels, scores),auc(recall, precision)

In [ ]:
'''
Decoder part. 
2 types of decoders.
Anyone u can use.
Source: DGL
'''

import dgl.function as fn
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h # here h is 822 drug features and g is the pos/neg train/test graph which is nothing but edge lis
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            return g.edata['score'][:, 0]

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']


decoder = MLPPredictor(128)
#decoder = DotPredictor()# You can replace DotPredictor with MLPPredictor.
#opt = torch.optim.Adam(list(model.parameters()) + list(pred.parameters()))

In [ ]:
'''
Encoder part (HAT)
'''
class HyperAttn_ne(nn.Module):
    # edge attention  version
    def __init__(self, input_dim, query_dim, vertex_dim, edge_dim, dropout):
        super(HyperAttn_ne, self).__init__()
        self.dropout = dropout
        
        self.query_dim = query_dim
        self.vtx_lin_1layer = torch.nn.Linear(input_dim, vertex_dim)
        self.vtx_lin = torch.nn.Linear(vertex_dim, vertex_dim)
        
        self.qe_lin = torch.nn.Linear(edge_dim, query_dim)
        self.kv_lin = torch.nn.Linear(vertex_dim, query_dim)
        self.vv_lin = torch.nn.Linear(vertex_dim, edge_dim)
        
        self.qv_lin = torch.nn.Linear(vertex_dim, query_dim)
        self.ke_lin = torch.nn.Linear(edge_dim, query_dim)
        self.ve_lin = torch.nn.Linear(edge_dim, vertex_dim)
        
        #self.cls = nn.Linear(vertex_dim, num_class)

    def attention(self, edges):
        attn_score = F.leaky_relu((edges.src['k'] * edges.dst['q']).sum(-1))
        return {'Attn': attn_score/np.sqrt(self.query_dim)}
    
    def message_func(self, edges):
        return {'v': edges.src['v'], 'Attn': edges.data['Attn']}

    def reduce_func(self, nodes):
        attention_score = F.softmax((nodes.mailbox['Attn']), dim=1)
        aggr = torch.sum(attention_score.unsqueeze(-1) * nodes.mailbox['v'], dim=1)
        return {'h': aggr}

    def forward(self, hyG, vfeat, efeat, first_layer, last_layer):
            if first_layer:
                feat_e = self.vtx_lin_1layer(efeat)
            else:
                feat_e = self.vtx_lin(efeat)        
            feat_v = vfeat

            # edge attention
            hyG.ndata['h'] = {'edge': feat_e}
            hyG.ndata['k'] = {'edge' : self.ke_lin(feat_e)}
            hyG.ndata['v'] = {'edge' : self.ve_lin(feat_e)}
            hyG.ndata['q'] = {'node' : self.qv_lin(feat_v)}
            hyG.apply_edges(self.attention, etype='con')
            hyG.update_all(self.message_func, self.reduce_func, etype='con')
            feat_v = hyG.ndata['h']['node']

            # node attention
            hyG.ndata['k'] = {'node' : self.kv_lin(feat_v)}
            hyG.ndata['v'] = {'node' : self.vv_lin(feat_v)}
            hyG.ndata['q'] = {'edge' : self.qe_lin(feat_e)}
            hyG.apply_edges(self.attention, etype='in')
            hyG.update_all(self.message_func, self.reduce_func, etype='in')
            feat_e = hyG.ndata['h']['edge']
            
            if not last_layer :
                feat_v = F.dropout(feat_v, self.dropout)
            if last_layer:
                #pred=self.cls(feat_v)
                return feat_v, feat_e
            else:
                return [hyG, feat_v, feat_e]

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.gat1 = HyperAttn_ne(drug_X.shape[1],64,128,128,0.5)
 
    def forward(self,hyG, v_feat, e_feat,f,l):   
        h = self.gat1(hyG, v_feat, e_feat,f,l)
        return h
model = Model()  

In [ ]:
# ----------- 3. set up loss and optimizer -------------- #
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), decoder.parameters()), lr=0.005)
best_val_loss=1e10
patience=0

for e in range(2000):
    # forward
    model.train()
    h=model(hyG, v_feat, e_feat,True,True)
    h=h[1]
    pos_score = decoder(train_pos_g, h)
    neg_score = decoder(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)
    
    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.no_grad():
      model.eval()
      pos_score = decoder(val_pos_g, h)
      neg_score = decoder(val_neg_g, h)
      #val_acc=compute_auc(pos_score, neg_score)
      val_loss = compute_loss(pos_score, neg_score)
      if val_loss<best_val_loss:
        best_val_loss=val_loss
        H=h
        E=e
        patience=0
        torch.save(decoder.state_dict(),'latest.pth')
      else:
        patience+=1  
    if patience>200:
      break

    if e % 10 == 0:
      print('In epoch {}, train loss: {:.4f}, val loss: {:.4f} (best val loss: {:.4f})'.format(e, loss, val_loss, best_val_loss))



In [ ]:
decoder.load_state_dict(torch.load('latest.pth'))
with torch.no_grad():
  model.eval()
  pos_score = decoder(test_pos_g, H)
  neg_score = decoder(test_neg_g, H)
  test_acc=compute_auc(pos_score, neg_score)

scores = torch.cat([pos_score, neg_score])
labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])

m1 = tf.keras.metrics.BinaryAccuracy()
m1.update_state(labels,scores)

sig_scores=F.sigmoid(scores)
m2 = tf.keras.metrics.Precision()
m2.update_state(labels,sig_scores)
M2=m2.result().numpy()

m3 = tf.keras.metrics.Recall()
m3.update_state(labels,sig_scores)
M3=m3.result().numpy()

F1=2*(M2*M3)/(M2+M3)
print(' Best Epoch: {}, Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score {:.4f}, ROC-AUC {:.4f}, PR-AUC {:.4f}'.format( E,m1.result().numpy(), M2, M3, F1,test_acc[0],test_acc[1]))

 Best Epoch: 934, Accuracy: 0.6843, Precision: 0.8296, Recall: 0.6478, F1-score 0.7275, ROC-AUC 0.7825, PR-AUC 0.8564


In [ ]:
##################################################### CASE STUDY #############################################

In [ ]:
lst=[]
for i in range (76760,76780): # range s ur choice 
  if sig_scores[i]<=0.00001: # use a large threshold say 0.9
    #print(sig_scores[i])
    lst.append(i)
print(len(lst))
sig_scores[lst[0]]

In [ ]:
c=0
for i in lst:
  j=i-76560
  print(sig_scores[i],test_neg_u[j],test_neg_v[j])
  for m,n in D.items():
    if n==test_neg_u[j]:
      print(m)
      break
  c+=1

In [ ]:
for i,j in D.items():
  if i=='DB01165':
    print(j)

1208


In [ ]:
# where we are cross-checking between what we predicted and what is in the DrugBank (label)
for i in range (len(mat)):
  for j in range (len(mat)):
    if i==410    and j==1621:
      print(mat[i][j])

0.0


In [ ]:
for i in range(len(test_neg_u)):
  if test_neg_u[i]==126 and test_neg_v[i]==338:
    print(i)

In [ ]:
for i in range (len(test_pos_u)):
  if test_pos_u[i]==656 and test_pos_v[i]==622:
    print(sig_scores[i])
  if test_pos_u[i]==622 and test_pos_v[i]==656:
    print(sig_scores[i])  

In [ ]:
********************************************************** GCN conv with ML **********************************

In [ ]:
'''
*************another approach
Train, test (both positive and negative) graphs creation for training and testing purpose from the dgl graph (g)
Source: DGL
'''
np.random.seed(42)
# Split edge set for training and testing
u, v = g.edges()
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
eids=eids.tolist()
test_size = int(len(eids) * 0.1)
train_size=len(eids)-(test_size)
test_size1=int(test_size/2)


T1=[]
T2=[]
numba.njit(target="cuda")
def func2(T1):
  for i in eids:
    m, n = int(u[i]), int(v[i])
    c=(m,n)
    T1.append(c)
  return T1  
function1=jit(parallel=True) (func2)
T1 = function1(T1)


numba.njit(target="cuda")
def func3(T1,T2):
  for i in T1:
    m=i[0]
    n=i[1]
    c=(m,n)
    d=(n,m)
    if c not in T2 and d not in T2:
      T2.append(c)
    if len(T2)>=test_size1:
      break  
  return T2
function1=jit(parallel=True) (func3)
T2 = function1(T1,T2)


test_eids=[]
numba.njit(target="cuda")
def func4(T2,test_eids,g):
  for i in T2:
    m= i[0]
    n=i[1]
    c=g.edge_id(m,n)
    d=g.edge_id(n,m)
    test_eids.append(c)
    test_eids.append(d)
  return test_eids  
function1=jit(parallel=True) (func4)
test_eids = function1(T2,test_eids,g)


train_eids=[]
numba.njit(target="cuda")
def func6(eids,test_eids,train_eids):
  i=0
  for j in eids:
    a=int(eids[i])
    if a not in test_eids  and len(train_eids)<train_size:
      train_eids.append(a)  
    i=i+1
  return train_eids    
function1=jit(parallel=True) (func6)
train_eids = function1(eids,test_eids,train_eids)     

# main one. splitting into train and test set.
test_pos_u, test_pos_v = u[test_eids], v[test_eids]
train_pos_u, train_pos_v = u[train_eids], v[train_eids]



# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges(),replace=False)
g_=nx.from_numpy_matrix(adj_neg)
g_=dgl.from_networkx(g_)

T1=[]
T2=[]
numba.njit(target="cuda")
def func2(T1):
  for i in neg_eids:
    m, n = int(neg_u[i]), int(neg_v[i])
    c=(m,n)
    T1.append(c)
  return T1  
function1=jit(parallel=True) (func2)
T1 = function1(T1)


numba.njit(target="cuda")
def func3(T1,T2):
  for i in T1:
    m=i[0]
    n=i[1]
    c=(m,n)
    d=(n,m)
    if c not in T2 and d not in T2:
      T2.append(c)
    if len(T2)>=test_size1:
      break  
  return T2
function1=jit(parallel=True) (func3)
T2 = function1(T1,T2)


test_neg_eids=[]
numba.njit(target="cuda")
def func4(T2,test_eids,g):
  for i in T2:
    m= i[0]
    n=i[1]
    c=g.edge_id(m,n)
    d=g.edge_id(n,m)
    test_neg_eids.append(c)
    test_neg_eids.append(d)
  return test_neg_eids  
function1=jit(parallel=True) (func4)
test_neg_eids = function1(T2,test_neg_eids,g_)


train_neg_eids=[]
numba.njit(target="cuda")
def func6(neg_eids,test_neg_eids,train_neg_eids):
  i=0
  for j in neg_eids:
    a=int(neg_eids[i])
    if a not in test_neg_eids and len(train_neg_eids)<(train_size):
      train_neg_eids.append(a)  
    i=i+1
  return train_neg_eids    
function1=jit(parallel=True) (func6)
train_neg_eids = function1(neg_eids,test_neg_eids,train_neg_eids)     

test_neg_u, test_neg_v = neg_u[test_neg_eids], neg_v[test_neg_eids]
train_neg_u, train_neg_v = neg_u[train_neg_eids], neg_v[train_neg_eids]


train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())# this is just the edge_list
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())
test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

print("Total number of edges: ",g.number_of_edges())
print("num of train POSITIVE edges: ",train_pos_g.number_of_edges(),", num of test POSITIVE edges: ",test_pos_g.number_of_edges())
print("num of train NEGATIVE edges: ",train_neg_g.number_of_edges(),", num of test NEGATIVE edges: ",test_neg_g.number_of_edges())

Total number of edges:  126946
num of train POSITIVE edges:  114252 , num of test POSITIVE edges:  12694
num of train NEGATIVE edges:  114252 , num of test NEGATIVE edges:  12694


In [ ]:
g_for_baseline = dgl.remove_edges(g, test_eids)
#dgl.save_graphs("drive/My Drive/Colab Notebooks/Hypergraph_my/a.bin",g_for_baseline)
g_for_baseline = dgl.add_self_loop(g_for_baseline)

In [ ]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
    def apply_edges(self, edges):
        h = torch.cat([edges.src['h'], edges.dst['h']], 1).squeeze(1)
        return {'score': h.squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']
decoder = MLPPredictor(128)

In [ ]:
from dgl.nn import GraphConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, h_feats)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

model = GCN(645, 128)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV

In [ ]:
e_feat = torch.zeros(645, 645)
e_feat.fill_diagonal_(1)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [ ]:
'''
when u want to use saved GCN embeddings
'''
h =np.loadtxt("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/afterTrainingEmbedsGCN.txt")
h=torch.tensor(h)
h=h.double()

In [ ]:
h = model(g_for_baseline, e_feat)    #uncomment it if u want to use original GCN from this notebook
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
modelNN = Sequential()
modelNN.add(Dense(10, input_dim=256, activation='sigmoid'))
modelNN.add(Dense(1, activation='sigmoid'))
modelNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelNN.fit(scores_train, labels_train, epochs=50, batch_size=20)
prediction_NN = modelNN.predict(scores_test)

print ('Accuracy:', accuracy_score(labels_test, prediction_NN.round()))
print ('F1 score:', f1_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Recall:', recall_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Precision:', precision_score(labels_test, prediction_NN.round(),average='weighted'))
print ('ROC-AUC:', roc_auc_score(labels_test, prediction_NN.round()))
precision, recall, thresholds = precision_recall_curve(labels_test, prediction_NN)
auc_precision_recall = auc(recall, precision)
print("PR-AUC: ",auc_precision_recall)

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

In [ ]:
********************************************************* Graph SAGE with ML ********************************************8

In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'pool')
        self.conv2 = SAGEConv(h_feats, h_feats, 'pool')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

model = GraphSAGE(645, 128)

In [ ]:
'''
when u want to use saved GCN embeddings
'''
h =np.loadtxt("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/afterTrainingEmbedsGS.txt")
h=torch.tensor(h)
h=h.double()

In [ ]:
h = model(g_for_baseline, e_feat)    #uncomment it if u want to use original GCN from this notebook
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
modelNN = Sequential()
modelNN.add(Dense(2, input_dim=200, activation='relu'))
modelNN.add(Dense(1, activation='sigmoid'))
modelNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelNN.fit(scores_train, labels_train, epochs=30, batch_size=10)
prediction_NN = modelNN.predict(scores_test)

print ('Accuracy:', accuracy_score(labels_test, prediction_NN.round()))
print ('F1 score:', f1_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Recall:', recall_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Precision:', precision_score(labels_test, prediction_NN.round(),average='weighted'))
print ('ROC-AUC:', roc_auc_score(labels_test, prediction_NN.round()))
precision, recall, thresholds = precision_recall_curve(labels_test, prediction_NN)
auc_precision_recall = auc(recall, precision)
print("PR-AUC: ",auc_precision_recall)

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.8580431700015756 precision:  0.8583509195434026 recall:  0.8580431700015756 f1-score:  0.8580126855458469 ROC-AUC:  0.8580431700015756 PR-AUC:  0.8920986304795103
NB
accuracy:  0.8037655585315897 precision:  0.8205412390813573 recall:  0.8037655585315897 f1-score:  0.8011640141958215 ROC-AUC:  0.8037655585315898 PR-AUC:  0.8531431672682562
RF
accuracy:  0.8103434693556011 precision:  0.8138989935092009 recall:  0.8103434693556011 f1-score:  0.8098048858172999 ROC-AUC:  0.8103434693556011 PR-AUC:  0.8561350355827504


In [ ]:
******************************************************************** GAT with ML ******************************************

In [ ]:
from dgl.nn import GATConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GAT(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_feats, h_feats, num_heads=1)
        self.conv2 = GATConv(h_feats, h_feats,num_heads=1)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

model = GAT(LEN, 128)

In [ ]:
'''
when u want to use saved GCN embeddings
'''
h =np.loadtxt("/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/afterTrainingEmbedsGAT.txt")
h=torch.tensor(h)
h=h.double()

In [ ]:
#h = model(g_for_baseline, e_feat)    #uncomment it if u want to use original GCN from this notebook
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
modelNN = Sequential()
modelNN.add(Dense(2, input_dim=200, activation='relu'))
modelNN.add(Dense(1, activation='sigmoid'))
modelNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelNN.fit(scores_train, labels_train, epochs=30, batch_size=10)
prediction_NN = modelNN.predict(scores_test)

print ('Accuracy:', accuracy_score(labels_test, prediction_NN.round()))
print ('F1 score:', f1_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Recall:', recall_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Precision:', precision_score(labels_test, prediction_NN.round(),average='weighted'))
print ('ROC-AUC:', roc_auc_score(labels_test, prediction_NN.round()))
precision, recall, thresholds = precision_recall_curve(labels_test, prediction_NN)
auc_precision_recall = auc(recall, precision)
print("PR-AUC: ",auc_precision_recall)

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.5437608318890814 precision:  0.5442210584531123 recall:  0.5437608318890814 f1-score:  0.5425706685547892 ROC-AUC:  0.5437608318890814 PR-AUC:  0.6685471815545543
NB
accuracy:  0.5205215062234126 precision:  0.5994697263323433 recall:  0.5205215062234126 f1-score:  0.40183122203828264 ROC-AUC:  0.5205215062234126 PR-AUC:  0.7469184485178153
RF
accuracy:  0.6097368835670396 precision:  0.6097372131784978 recall:  0.6097368835670396 f1-score:  0.6097365905140091 ROC-AUC:  0.6097368835670396 PR-AUC:  0.7074243723569373


In [ ]:
************************************************************************ GIN with ML ***************************************************888888

In [ ]:
from dgl.nn import GINConv
lin1 = torch.nn.Linear(824, 100)
lin2 = torch.nn.Linear(100, 100)
# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GIN(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GIN, self).__init__()
        self.conv1 = GINConv(lin1, 'max')
        self.conv2 = GINConv(lin2, 'max')
        
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

model = GIN(824, 100)

In [ ]:
h = model(g_for_baseline, e_feat)
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)


scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
modelNN = Sequential()
modelNN.add(Dense(2, input_dim=200, activation='relu'))
modelNN.add(Dense(1, activation='sigmoid'))
modelNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelNN.fit(scores_train, labels_train, epochs=30, batch_size=10)
prediction_NN = modelNN.predict(scores_test)

print ('Accuracy:', accuracy_score(labels_test, prediction_NN.round()))
print ('F1 score:', f1_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Recall:', recall_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Precision:', precision_score(labels_test, prediction_NN.round(),average='weighted'))
print ('ROC-AUC:', roc_auc_score(labels_test, prediction_NN.round()))
precision, recall, thresholds = precision_recall_curve(labels_test, prediction_NN)
auc_precision_recall = auc(recall, precision)
print("PR-AUC: ",auc_precision_recall)

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


In [ ]:
************************************************************************ AGNNConv with ML ***************************************************888888

In [ ]:
from dgl.nn import AGNNConv
# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class AGNN(nn.Module):
    def __init__(self):
        super(AGNN, self).__init__()
        self.conv1 = AGNNConv()
        self.conv2 = AGNNConv()
          
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

model = AGNN()

In [ ]:
h = model(g_for_baseline, e_feat)
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
modelNN = Sequential()
modelNN.add(Dense(2, input_dim=1648, activation='relu'))
modelNN.add(Dense(1, activation='sigmoid'))
modelNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelNN.fit(scores_train, labels_train, epochs=30, batch_size=10)
prediction_NN = modelNN.predict(scores_test)

print ('Accuracy:', accuracy_score(labels_test, prediction_NN.round()))
print ('F1 score:', f1_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Recall:', recall_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Precision:', precision_score(labels_test, prediction_NN.round(),average='weighted'))
print ('ROC-AUC:', roc_auc_score(labels_test, prediction_NN.round()))
precision, recall, thresholds = precision_recall_curve(labels_test, prediction_NN)
auc_precision_recall = auc(recall, precision)
print("PR-AUC: ",auc_precision_recall)

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


In [ ]:
***************************************************************** DeepWalk **********************************

In [ ]:
g_for_baseline_adj_matrix=g_for_baseline.adjacency_matrix_scipy()
G_baseline = nx.from_numpy_array(g_for_baseline_adj_matrix,create_using=nx.DiGraph)
nx.write_edgelist(G_baseline, "/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/G_baseline.txt", data=False)

# train model and generate embedding
model = DeepWalk(walk_length=100, dimensions=128, window_size=5)
model.fit(G_baseline)
h = model.get_embedding()
h=torch.from_numpy(h)

In [ ]:
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
modelNN = Sequential()
modelNN.add(Dense(2, input_dim=200, activation='relu'))
modelNN.add(Dense(1, activation='sigmoid'))
modelNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelNN.fit(scores_train, labels_train, epochs=30, batch_size=10)
prediction_NN = modelNN.predict(scores_test)

print ('Accuracy:', accuracy_score(labels_test, prediction_NN.round()))
print ('F1 score:', f1_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Recall:', recall_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Precision:', precision_score(labels_test, prediction_NN.round(),average='weighted'))
print ('ROC-AUC:', roc_auc_score(labels_test, prediction_NN.round()))
precision, recall, thresholds = precision_recall_curve(labels_test, prediction_NN)
auc_precision_recall = auc(recall, precision)
print("PR-AUC: ",auc_precision_recall)

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.803608003781314 precision:  0.8040037464734512 recall:  0.803608003781314 f1-score:  0.8035440687146947 ROC-AUC:  0.803608003781314 PR-AUC:  0.8519296455746475
NB
accuracy:  0.8127461792973059 precision:  0.8129760221290243 recall:  0.8127461792973059 f1-score:  0.8127117941938743 ROC-AUC:  0.8127461792973059 PR-AUC:  0.8588212491450073
RF
accuracy:  0.8079801481014652 precision:  0.8085828471645349 recall:  0.8079801481014652 f1-score:  0.8078863428865861 ROC-AUC:  0.8079801481014655 PR-AUC:  0.8549919522864071


In [ ]:
******************************************************************* Node2Vec ***********************************

In [ ]:
h=np.loadtxt('/content/drive/MyDrive/My Drive/Colab Notebooks/Hypergraph_my/data/node_2vec_embeddings.txt',delimiter=',')
h=torch.from_numpy(h)

In [ ]:
pos_score_train = decoder(train_pos_g, h)
neg_score_train = decoder(train_neg_g, h)
pos_score_test = decoder(test_pos_g, h)
neg_score_test = decoder(test_neg_g, h)

scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
modelNN = Sequential()
modelNN.add(Dense(2, input_dim=200, activation='relu'))
modelNN.add(Dense(1, activation='sigmoid'))
modelNN.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
modelNN.fit(scores_train, labels_train, epochs=30, batch_size=10)
prediction_NN = modelNN.predict(scores_test)

print ('Accuracy:', accuracy_score(labels_test, prediction_NN.round()))
print ('F1 score:', f1_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Recall:', recall_score(labels_test, prediction_NN.round(),average='weighted'))
print ('Precision:', precision_score(labels_test, prediction_NN.round(),average='weighted'))
print ('ROC-AUC:', roc_auc_score(labels_test, prediction_NN.round()))
precision, recall, thresholds = precision_recall_curve(labels_test, prediction_NN)
auc_precision_recall = auc(recall, precision)
print("PR-AUC: ",auc_precision_recall)

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions,average='weighted'),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.8432724121632267 precision:  0.8435291119502388 recall:  0.8432724121632267 f1-score:  0.8432431282872891 ROC-AUC:  0.8432724121632268 PR-AUC:  0.8813043148436112
NB
accuracy:  0.8139672286119426 precision:  0.8140282832006075 recall:  0.8139672286119426 f1-score:  0.8139581858703779 ROC-AUC:  0.8139672286119426 PR-AUC:  0.8600595565157555
RF
accuracy:  0.7811958405545927 precision:  0.7820122582432303 recall:  0.7811958405545927 f1-score:  0.7810373678634227 ROC-AUC:  0.7811958405545928 PR-AUC:  0.8354438803276918


In [ ]:
************************************************************************ Unsupervised GNNs ******************************************************************

In [ ]:
import sys
import math
import copy
import random
from collections import defaultdict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from scipy.sparse import csr_matrix

class GNN(object):
    """Graph Neural Networks that can be easily called and used.
    Authors of this code package:
    Tong Zhao, tzhao2@nd.edu
    Tianwen Jiang, twjiang@ir.hit.edu.cn
    Last updated: 11/25/2019
    Parameters
    ----------
    adj_matrix: scipy.sparse.csr_matrix
        The adjacency matrix of the graph, where nonzero entries indicates edges.
        The number of each nonzero entry indicates the number of edges between these two nodes.
    features: numpy.ndarray, optional
        The 2-dimension np array that stores given raw feature of each node, where the i-th row
        is the raw feature vector of node i.
        When raw features are not given, one-hot degree features will be used.
    labels: list or 1-D numpy.ndarray, optional
        The class label of each node. Used for supervised learning.
    supervised: bool, optional, default False
        Whether to use supervised learning.
    model: {'gat', 'graphsage'}, default 'gat'
        The GNN model to be used.
        - 'graphsage' is GraphSAGE: https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf
        - 'gat' is graph attention network: https://arxiv.org/pdf/1710.10903.pdf
    n_layer: int, optional, default 2
        Number of layers in the GNN
    emb_size: int, optional, default 128
        Size of the node embeddings to be learnt
    random_state, int, optional, default 1234
        Random seed
    device: {'cpu', 'cuda', 'auto'}, default 'auto'
        The device to use.
    epochs: int, optional, default 5
        Number of epochs for training
    batch_size: int, optional, default 20
        Number of node per batch for training
    lr: float, optional, default 0.7
        Learning rate
    unsup_loss_type: {'margin', 'normal'}, default 'margin'
        Loss function to be used for unsupervised learning
        - 'margin' is a hinge loss with margin of 3
        - 'normal' is the unsupervised loss function described in the paper of GraphSAGE
    print_progress: bool, optional, default True
        Whether to print the training progress
    """
    def __init__(self, adj_matrix, features=None, labels=None, supervised=False, model='gat', n_layer=2, emb_size=128, random_state=1234, device='auto', epochs=5, batch_size=20, lr=0.7, unsup_loss_type='margin', print_progress=True):
        super(GNN, self).__init__()
        # fix random seeds
        random.seed(random_state)
        np.random.seed(random_state)
        torch.manual_seed(random_state)
        torch.cuda.manual_seed_all(random_state)
        # set parameters
        self.supervised = supervised
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.unsup_loss_type = unsup_loss_type
        self.print_progress = print_progress
        self.gat = False
        self.gcn = False
        if model == 'gat':
            self.gat = True
            self.model_name = 'GAT'
        elif model == 'gcn':
            self.gcn = True
            self.model_name = 'GCN'
        else:
            self.model_name = 'GraphSAGE'
        # set device
        if device == 'auto':
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device

        # load data
        self.dl = DataLoader(adj_matrix, features, labels, supervised, self.device)

        self.gnn = GNN_model(n_layer, emb_size, self.dl, self.device, gat=self.gat, gcn=self.gcn)
        self.gnn.to(self.device)

        if supervised:
            n_classes = len(set(labels))
            self.classification = Classification(emb_size, n_classes)
            self.classification.to(self.device)

    def fit(self):
        train_nodes = copy.deepcopy(self.dl.nodes_train)

        if self.supervised:
            labels = self.dl.labels
            models = [self.gnn, self.classification]
        else:
            unsup_loss = Unsup_Loss(self.dl, self.device)
            models = [self.gnn]
            if self.unsup_loss_type == 'margin':
                num_neg = 6
            elif self.unsup_loss_type == 'normal':
                num_neg = 100

        for epoch in range(self.epochs):
            np.random.shuffle(train_nodes)

            params = []
            for model in models:
                for param in model.parameters():
                    if param.requires_grad:
                        params.append(param)
            optimizer = torch.optim.SGD(params, lr=self.lr)
            optimizer.zero_grad()
            for model in models:
                model.zero_grad()

            batches = math.ceil(len(train_nodes) / self.batch_size)
            visited_nodes = set()
            if self.print_progress:
                tqdm_bar = tqdm(range(batches), ascii=True, leave=False)
            else:
                tqdm_bar = range(batches)
            for index in tqdm_bar:
                if not self.supervised and len(visited_nodes) == len(train_nodes):
                    # finish this epoch if all nodes are visited
                    if self.print_progress:
                        tqdm_bar.close()
                    break
                nodes_batch = train_nodes[index*self.batch_size:(index+1)*self.batch_size]
                # extend nodes batch for unspervised learning
                if not self.supervised:
                    nodes_batch = np.asarray(list(unsup_loss.extend_nodes(nodes_batch, num_neg=num_neg)))
                visited_nodes |= set(nodes_batch)
                # feed nodes batch to the GNN and returning the nodes embeddings
                embs_batch = self.gnn(nodes_batch)
                # calculate loss
                if self.supervised:
                    # superivsed learning
                    logists = self.classification(embs_batch)
                    labels_batch = labels[nodes_batch]
                    loss_sup = -torch.sum(logists[range(logists.size(0)), labels_batch], 0)
                    loss_sup /= len(nodes_batch)
                    loss = loss_sup
                else:
                    # unsupervised learning
                    if self.unsup_loss_type == 'margin':
                        loss_net = unsup_loss.get_loss_margin(embs_batch, nodes_batch)
                    elif self.unsup_loss_type == 'normal':
                        loss_net = unsup_loss.get_loss_sage(embs_batch, nodes_batch)
                    loss = loss_net

                if self.print_progress:
                    progress_message = '{} Epoch: [{}/{}], current loss: {:.4f}, touched nodes [{}/{}] '.format(
                                    self.model_name, epoch+1, self.epochs, loss.item(), len(visited_nodes), len(train_nodes))
                    tqdm_bar.set_description(progress_message)

                loss.backward()
                for model in models:
                    nn.utils.clip_grad_norm_(model.parameters(), 5)
                optimizer.step()
                optimizer.zero_grad()
                for model in models:
                    model.zero_grad()

    def generate_embeddings(self):
        nodes = self.dl.nodes_train
        b_sz = 500
        batches = math.ceil(len(nodes) / b_sz)
        embs = []
        for index in range(batches):
            nodes_batch = nodes[index*b_sz:(index+1)*b_sz]
            with torch.no_grad():
                embs_batch = self.gnn(nodes_batch)
            assert len(embs_batch) == len(nodes_batch)
            embs.append(embs_batch)
        assert len(embs) == batches
        embs = torch.cat(embs, 0)
        assert len(embs) == len(nodes)
        return embs.cpu().numpy()

    def predict(self):
        if not self.supervised:
            print('GNN.predict() is only supported for supervised learning.')
            sys.exit(0)
        nodes = self.dl.nodes_train
        b_sz = 500
        batches = math.ceil(len(nodes) / b_sz)
        preds = []
        for index in range(batches):
            nodes_batch = nodes[index*b_sz:(index+1)*b_sz]
            with torch.no_grad():
                embs_batch = self.gnn(nodes_batch)
                logists = self.classification(embs_batch)
                _, predicts = torch.max(logists, 1)
                preds.append(predicts)
        assert len(preds) == batches
        preds = torch.cat(preds, 0)
        assert len(preds) == len(nodes)
        return preds.cpu().numpy()

    def release_cuda_cache(self):
        torch.cuda.empty_cache()


class DataLoader(object):
    def __init__(self, adj_matrix, raw_features, labels, supervised, device):
        super(DataLoader, self).__init__()
        self.adj_matrix = adj_matrix
        # load adjacency list and node features
        self.adj_list = self.get_adj_list(adj_matrix)
        if raw_features is None:
            features = self.get_features()
        else:
            features = raw_features
        assert features.shape[0] == len(self.adj_list) == self.adj_matrix.shape[0]
        self.features = torch.FloatTensor(features).to(device)
        self.nodes_train = list(range(len(self.adj_list)))
        if supervised:
            self.labels = np.asarray(labels)

    def get_adj_list(self, adj_matrix):
        """build adjacency list from adjacency matrix"""
        adj_list = {}
        for i in range(adj_matrix.shape[0]):
            adj_list[i] = set(np.where(adj_matrix[i].toarray() != 0)[1])
        return adj_list

    def get_features(self):
        """
        When raw features are not available,
        build one-hot degree features from the adjacency list.
        """
        max_degree = np.max(np.sum(self.adj_matrix != 0, axis=1))
        features = np.zeros((self.adj_matrix.shape[0], max_degree))
        for node, neighbors in self.adj_list.items():
            features[node, len(neighbors)-1] = 1
        return features


class Classification(nn.Module):
    def __init__(self, emb_size, num_classes):
        super(Classification, self).__init__()
        self.fc1 = nn.Linear(emb_size, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, embeds):
        x = F.elu(self.fc1(embeds))
        x = F.elu(self.fc2(x))
        logists = torch.log_softmax(x, 1)
        return logists


class Unsup_Loss(object):
    """docstring for UnsupervisedLoss"""
    def __init__(self, dl, device):
        super(Unsup_Loss, self).__init__()
        self.Q = 10
        self.N_WALKS = 4
        self.WALK_LEN = 4
        self.N_WALK_LEN = 5
        self.MARGIN = 3
        self.adj_lists = dl.adj_list
        self.adj_matrix = dl.adj_matrix
        self.train_nodes = dl.nodes_train
        self.device = device

        self.target_nodes = None
        self.positive_pairs = []
        self.negative_pairs = []
        self.node_positive_pairs = {}
        self.node_negative_pairs = {}
        self.unique_nodes_batch = []

    def get_loss_sage(self, embeddings, nodes):
        assert len(embeddings) == len(self.unique_nodes_batch)
        assert False not in [nodes[i]==self.unique_nodes_batch[i] for i in range(len(nodes))]
        node2index = {n:i for i,n in enumerate(self.unique_nodes_batch)}

        nodes_score = []
        assert len(self.node_positive_pairs) == len(self.node_negative_pairs)
        for node in self.node_positive_pairs:
            pps = self.node_positive_pairs[node]
            nps = self.node_negative_pairs[node]
            if len(pps) == 0 or len(nps) == 0:
                continue

            # Q * Exception(negative score)
            indexs = [list(x) for x in zip(*nps)]
            node_indexs = [node2index[x] for x in indexs[0]]
            neighb_indexs = [node2index[x] for x in indexs[1]]
            neg_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
            neg_score = self.Q*torch.mean(torch.log(torch.sigmoid(-neg_score)), 0)

            # multiple positive score
            indexs = [list(x) for x in zip(*pps)]
            node_indexs = [node2index[x] for x in indexs[0]]
            neighb_indexs = [node2index[x] for x in indexs[1]]
            pos_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
            pos_score = torch.log(torch.sigmoid(pos_score))

            nodes_score.append(torch.mean(- pos_score - neg_score).view(1,-1))

        loss = torch.mean(torch.cat(nodes_score, 0))
        return loss

    def get_loss_margin(self, embeddings, nodes):
        
        '''I commented these out bc it was throwing assertion error that I can't understand '''
#         assert len(embeddings) == len(self.unique_nodes_batch)
#         assert False not in [nodes[i]==self.unique_nodes_batch[i] for i in range(len(nodes))]


        node2index = {n:i for i,n in enumerate(self.unique_nodes_batch)}

        nodes_score = []
        
        
        ''' See above'''
#         assert len(self.node_positive_pairs) == len(self.node_negative_pairs) 
        for node in self.node_positive_pairs:
            pps = self.node_positive_pairs[node]
            nps = self.node_negative_pairs[node]
            if len(pps) == 0 or len(nps) == 0:
                continue

            indexs = [list(x) for x in zip(*pps)]
            node_indexs = [node2index[x] for x in indexs[0]]
            neighb_indexs = [node2index[x] for x in indexs[1]]
            pos_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
            pos_score, _ = torch.min(torch.log(torch.sigmoid(pos_score)), 0)

            indexs = [list(x) for x in zip(*nps)]
            node_indexs = [node2index[x] for x in indexs[0]]
            neighb_indexs = [node2index[x] for x in indexs[1]]
            neg_score = F.cosine_similarity(embeddings[node_indexs], embeddings[neighb_indexs])
            neg_score, _ = torch.max(torch.log(torch.sigmoid(neg_score)), 0)

            nodes_score.append(torch.max(torch.tensor(0.0).to(self.device),
                                         neg_score-pos_score+self.MARGIN).view(1, -1))
        loss = torch.mean(torch.cat(nodes_score, 0), 0)
        return loss

    def extend_nodes(self, nodes, num_neg=6):
        self.positive_pairs = []
        self.node_positive_pairs = {}
        self.negative_pairs = []
        self.node_negative_pairs = {}

        self.target_nodes = nodes
        self.get_positive_nodes(nodes)
        self.get_negative_nodes(nodes, num_neg)
        self.unique_nodes_batch = list(set([i for x in self.positive_pairs for i in x])
                                       | set([i for x in self.negative_pairs for i in x]))
        assert set(self.target_nodes) < set(self.unique_nodes_batch)
        return self.unique_nodes_batch

    def get_positive_nodes(self, nodes):
        return self._run_random_walks(nodes)

    def get_negative_nodes(self, nodes, num_neg):
        for node in nodes:
            neighbors = set([node])
            frontier = set([node])
            for _ in range(self.N_WALK_LEN):
                current = set()
                for outer in frontier:
                    current |= self.adj_lists[int(outer)]
                frontier = current - neighbors
                neighbors |= current
            far_nodes = set(self.train_nodes) - neighbors
            neg_samples = random.sample(far_nodes, num_neg) if num_neg < len(far_nodes) else far_nodes
            self.negative_pairs.extend([(node, neg_node) for neg_node in neg_samples])
            self.node_negative_pairs[node] = [(node, neg_node) for neg_node in neg_samples]
        return self.negative_pairs

    def _run_random_walks(self, nodes):
        for node in nodes:
            if len(self.adj_lists[int(node)]) == 0:
                continue
            cur_pairs = []
            for _ in range(self.N_WALKS):
                curr_node = node
                for _ in range(self.WALK_LEN):
                    cnts = self.adj_matrix[int(curr_node)].toarray().squeeze()
                    neighs = []
                    for n in np.where(cnts != 0)[0]:
                        neighs.extend([n] * int(cnts[n]))
                    # neighs = self.adj_lists[int(curr_node)]
                    next_node = random.choice(list(neighs))
                    # self co-occurrences are useless
                    if next_node != node and next_node in self.train_nodes:
                        self.positive_pairs.append((node,next_node))
                        cur_pairs.append((node,next_node))
                    curr_node = next_node

            self.node_positive_pairs[node] = cur_pairs
        return self.positive_pairs


class SageLayer(nn.Module):
    """
    Encodes a node's using 'convolutional' GraphSage approach
    """
    def __init__(self, input_size, out_size, gat=False, gcn=False):
        super(SageLayer, self).__init__()

        self.input_size = input_size
        self.out_size = out_size

        self.gat = gat
        self.gcn = gcn
        self.weight = nn.Parameter(torch.FloatTensor(out_size, self.input_size if self.gat or self.gcn else 2 * self.input_size))

        self.init_params()

    def init_params(self):
        for param in self.parameters():
            nn.init.xavier_uniform_(param)

    def forward(self, self_feats, aggregate_feats):
        """
        Generates embeddings for a batch of nodes.
        nodes	 -- list of nodes
        """
        if self.gat or self.gcn:
            combined = aggregate_feats
        else:
            combined = torch.cat([self_feats, aggregate_feats], dim=1)
        combined = F.relu(self.weight.mm(combined.t())).t()
        return combined

class Attention(nn.Module):
    """Computes the self-attention between pair of nodes"""
    def __init__(self, input_size, out_size):
        super(Attention, self).__init__()

        self.input_size = input_size
        self.out_size = out_size
        self.attention_raw = nn.Linear(2*input_size, 1, bias=False)
        self.attention_emb = nn.Linear(2*out_size, 1, bias=False)

    def forward(self, row_embs, col_embs):
        if row_embs.size(1) == self.input_size:
            att = self.attention_raw
        elif row_embs.size(1) == self.out_size:
            att = self.attention_emb
        e = att(torch.cat((row_embs, col_embs), dim=1))
        return F.leaky_relu(e, negative_slope=0.2)

class GNN_model(nn.Module):
    """docstring for GraphSage"""
    def __init__(self, num_layers, out_size, dl, device, gat=False, gcn=False, agg_func='MEAN'):
        super(GNN_model, self).__init__()

        self.input_size = dl.features.size(1)
        self.out_size = out_size
        self.num_layers = num_layers
        self.gat = gat
        self.gcn = gcn
        self.device = device
        self.agg_func = agg_func

        self.raw_features = dl.features
        self.adj_lists = dl.adj_list
        self.adj_matrix = dl.adj_matrix

        for index in range(1, num_layers+1):
            layer_size = out_size if index != 1 else self.input_size
            setattr(self, 'sage_layer'+str(index), SageLayer(layer_size, out_size, gat=self.gat, gcn=self.gcn))
        if self.gat:
            self.attention = Attention(self.input_size, out_size)

    def forward(self, nodes_batch):
        """
        Generates embeddings for a batch of nodes.
        nodes_batch	-- batch of nodes to learn the embeddings
        """
        lower_layer_nodes = list(nodes_batch)
        nodes_batch_layers = [(lower_layer_nodes,)]
        for _ in range(self.num_layers):
            lower_layer_nodes, lower_samp_neighs, lower_layer_nodes_dict= self._get_unique_neighs_list(lower_layer_nodes)
            nodes_batch_layers.insert(0, (lower_layer_nodes, lower_samp_neighs, lower_layer_nodes_dict))

        assert len(nodes_batch_layers) == self.num_layers + 1

        pre_hidden_embs = self.raw_features
        for index in range(1, self.num_layers+1):
            nb = nodes_batch_layers[index][0]
            pre_neighs = nodes_batch_layers[index-1]
            aggregate_feats = self.aggregate(nb, pre_hidden_embs, pre_neighs)
            sage_layer = getattr(self, 'sage_layer'+str(index))
            if index > 1:
                nb = self._nodes_map(nb, pre_neighs)
            cur_hidden_embs = sage_layer(self_feats=pre_hidden_embs[nb], aggregate_feats=aggregate_feats)
            pre_hidden_embs = cur_hidden_embs

        return pre_hidden_embs

    def _nodes_map(self, nodes, neighs):
        _, samp_neighs, layer_nodes_dict = neighs
        assert len(samp_neighs) == len(nodes)
        index = [layer_nodes_dict[x] for x in nodes]
        return index

    def _get_unique_neighs_list(self, nodes, num_sample=10):
        _set = set
        to_neighs = [self.adj_lists[int(node)] for node in nodes]
        if self.gcn or self.gat:
            samp_neighs = to_neighs
        else:
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh, num_sample)) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
        samp_neighs = [samp_neigh | set([nodes[i]]) for i, samp_neigh in enumerate(samp_neighs)]
        _unique_nodes_list = list(set.union(*samp_neighs))
        i = list(range(len(_unique_nodes_list)))
        # unique node 2 index
        unique_nodes = dict(list(zip(_unique_nodes_list, i)))
        return _unique_nodes_list, samp_neighs, unique_nodes

    def aggregate(self, nodes, pre_hidden_embs, pre_neighs):
        unique_nodes_list, samp_neighs, unique_nodes = pre_neighs

        assert len(nodes) == len(samp_neighs)
        indicator = [(nodes[i] in samp_neighs[i]) for i in range(len(samp_neighs))]
        assert False not in indicator
        if not self.gat and not self.gcn:
            samp_neighs = [(samp_neighs[i]-set([nodes[i]])) for i in range(len(samp_neighs))]
        if len(pre_hidden_embs) == len(unique_nodes):
            embed_matrix = pre_hidden_embs
        else:
            embed_matrix = pre_hidden_embs[torch.LongTensor(unique_nodes_list)]
        # get row and column nonzero indices for the mask tensor
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]
        # get the edge counts for each edge
        edge_counts = self.adj_matrix[nodes][:, unique_nodes_list].toarray()
        edge_counts = torch.FloatTensor(edge_counts).to(embed_matrix.device)
        torch.sqrt_(edge_counts)
        if self.gat:
            indices = (torch.LongTensor(row_indices), torch.LongTensor(column_indices))
            nodes_indices = torch.LongTensor([unique_nodes[nodes[n]] for n in row_indices])
            row_embs = embed_matrix[nodes_indices]
            col_embs = embed_matrix[column_indices]
            atts = self.attention(row_embs, col_embs).squeeze()
            mask = torch.zeros(len(samp_neighs), len(unique_nodes)).to(embed_matrix.device)
            mask.index_put_(indices, atts)
            mask = mask * edge_counts
            # softmax
            mask = torch.exp(mask) * (mask != 0).float()
            mask = F.normalize(mask, p=1, dim=1)
        else:
            mask = torch.zeros(len(samp_neighs), len(unique_nodes)).to(embed_matrix.device)
            mask[row_indices, column_indices] = 1
            # multiply edge counts to mask
            mask = mask * edge_counts
            mask = F.normalize(mask, p=1, dim=1)
            mask = mask.to(embed_matrix.device)

        if self.agg_func == 'MEAN':
            aggregate_feats = mask.mm(embed_matrix)
        elif self.agg_func == 'MAX':
            indexs = [x.nonzero() for x in mask != 0]
            aggregate_feats = []
            for feat in [embed_matrix[x.squeeze()] for x in indexs]:
                if len(feat.size()) == 1:
                    aggregate_feats.append(feat.view(1, -1))
                else:
                    aggregate_feats.append(torch.max(feat,0)[0].view(1, -1))
            aggregate_feats = torch.cat(aggregate_feats, 0)

        return aggregate_feats

In [ ]:

gcn =GNN(line_Adj, epochs=5, supervised=False, model='gcn')
graphsage =GNN(line_Adj, epochs=5, supervised=False, model='graphsage')
gat =GNN(line_Adj, epochs=5, supervised=False, model='gat')

gcn.fit()
graphsage.fit()
gat.fit()

embs_gcn = gcn.generate_embeddings()
embs_graphsage = graphsage.generate_embeddings()
embs_gat = gat.generate_embeddings()

In [ ]:
embs_gcn=torch.from_numpy(embs_gcn)
embs_graphsage=torch.from_numpy(embs_graphsage)
embs_gat=torch.from_numpy(embs_gat)

In [ ]:
pos_score_train = decoder(train_pos_g, embs_gcn)
neg_score_train = decoder(train_neg_g, embs_gcn)
pos_score_test = decoder(test_pos_g, embs_gcn)
neg_score_test = decoder(test_neg_g, embs_gcn)


scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
scores_test.shape

(38700, 256)

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.541266149870801 precision:  0.5508851764272542 recall:  0.541266149870801 f1-score:  0.6231825610765606 ROC-AUC:  0.5412661498708009 PR-AUC:  0.7040447331934078
NB
accuracy:  0.5448837209302325 precision:  0.5902227884768768 recall:  0.5448837209302325 f1-score:  0.6639830589312627 ROC-AUC:  0.5448837209302325 PR-AUC:  0.7379644694441925
RF
accuracy:  0.546046511627907 precision:  0.5846158397636402 recall:  0.546046511627907 f1-score:  0.6606135538211885 ROC-AUC:  0.546046511627907 PR-AUC:  0.7346484539729777


In [ ]:
pos_score_train = decoder(train_pos_g, embs_graphsage)
neg_score_train = decoder(train_neg_g, embs_graphsage)
pos_score_test = decoder(test_pos_g, embs_graphsage)
neg_score_test = decoder(test_neg_g, embs_graphsage)


scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
print("LR")
lr = LogisticRegression()
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy:  0.5731524547803618 precision:  0.5733092898367258 recall:  0.5731524547803618 f1-score:  0.5828008586942797 ROC-AUC:  0.5731524547803618 PR-AUC:  0.6840290166391932
NB
accuracy:  0.5630232558139535 precision:  0.5725284939049564 recall:  0.5630232558139535 f1-score:  0.6299967180833607 ROC-AUC:  0.5630232558139534 PR-AUC:  0.7091437792572788
RF
accuracy:  0.5724289405684755 precision:  0.5778336918858435 recall:  0.5724289405684755 f1-score:  0.6222059864380465 ROC-AUC:  0.5724289405684754 PR-AUC:  0.704708215583079


In [ ]:
pos_score_train = decoder(train_pos_g, embs_gat)
neg_score_train = decoder(train_neg_g, embs_gat)
pos_score_test = decoder(test_pos_g, embs_gat)
neg_score_test = decoder(test_neg_g, embs_gat)


scores_train = torch.cat([pos_score_train, neg_score_train]).detach().numpy()
labels_train = torch.cat([torch.ones(pos_score_train.shape[0]), torch.zeros(neg_score_train.shape[0])]).detach().numpy()
scores_test = torch.cat([pos_score_test, neg_score_test]).detach().numpy()
labels_test = torch.cat([torch.ones(pos_score_test.shape[0]), torch.zeros(neg_score_test.shape[0])]).detach().numpy()

In [ ]:
print("LR")
lr = LogisticRegression(class_weight="balanced")
lr.fit(scores_train, labels_train)
predictions = lr.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("NB")
gnb = GaussianNB()
gnb.fit(scores_train, labels_train)
predictions = gnb.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)

print("RF")
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(scores_train, labels_train)
predictions = clf.predict(scores_test)
precision, recall, thresholds = precision_recall_curve(labels_test, predictions)
auc_precision_recall = auc(recall, precision)
print("accuracy: ",accuracy_score(labels_test,predictions),"precision: ",precision_score(labels_test,predictions,average='weighted'),'recall: ',recall_score(labels_test,predictions,average='weighted'),'f1-score: ',f1_score(labels_test,predictions),'ROC-AUC: ',roc_auc_score(labels_test, predictions),'PR-AUC: ',auc_precision_recall)


LR
accuracy:  0.538062015503876 precision:  0.5466821348139691 recall:  0.538062015503876 f1-score:  0.6197596511751569 ROC-AUC:  0.538062015503876 PR-AUC:  0.7015410169941123
NB
accuracy:  0.5384496124031007 precision:  0.5715021618621324 recall:  0.5384496124031007 f1-score:  0.6555461277383523 ROC-AUC:  0.5384496124031007 PR-AUC:  0.731043524304239
RF
accuracy:  0.5459689922480621 precision:  0.5869585920668676 recall:  0.5459689922480621 f1-score:  0.6619986534577282 ROC-AUC:  0.545968992248062 PR-AUC:  0.7359406682399366


In [ ]:
precision

array([0.5       , 0.52590692, 1.        ])

In [ ]:
'''
here, the hyG creation starts
1. at frist we read the text file "new_graph_data_1.txt". This text file is created by running ESPF algorithm
2. Then we create a dictionary 'DICT' that has 'drug' as key and drug's 'substructures' as values 
'''
formattedNodes = []
smileFile = open("drive/My Drive/Colab Notebooks/Hypergraph_my/new_graph_data.txt", 'r')
names = []

for line in smileFile:
    line = line.strip()
    words = line.split(" ")
    name = words[0]
    del words[0]
    t=tuple(words)
    names.append(name)
    formattedNodes.append(t)
      
# Make the dictionary of the Drug IDs: Smiles tokens    
DICT = dict()  
for i in range(0, len(names)):
    DICT[names[i]] = formattedNodes[i]    



In [ ]:
# Pass the dict to create graph
H = hnx.Hypergraph(DICT)
incidence=H.incidence_matrix()
"""
This below line graph/regular graph is generated from the hyG that will be used in GCN
"""
H1 = H.convert_to_static(H)
Hline = H1.get_linegraph(s = 2, edges=True, use_nwhy=True)
line_Adj = nx.adjacency_matrix(Hline)

In [ ]:
line_Adj = nx.to_scipy_sparse_matrix(Hline, format = 'csr')
line_Adj=line_Adj.toarray()
np.fill_diagonal(line_Adj, 1)

In [ ]:
from scipy import sparse
line_Adj= sparse.csr_matrix(line_Adj) 